In [ ]:
import numpy as np
import pandas
import pm4py
from matplotlib import pyplot as plt
from sklearn.mixture import GaussianMixture
import scipy.stats as stats
import ot
import os
from tqdm import tqdm
import collections
import matplotlib.dates as md
import importlib
import pickle
import random
import math
import CRPS.CRPS as pscore
import datetime

pandas.set_option('display.max_columns', None)
#pandas.set_option('display.max_rows', None)


import sys
sys.path.append('../../TaskExecutionTimeMining/')
from drbart_parser import *
from event_log_transformer import *

#sys.path.append('../../Evaluation')
sys.path.append('../../Evaluation/')
from PCR_evaluation.drbart_evaluation import *

In [ ]:
log_name = 'train'
with open('../transformed_event_logs/PCR_start_end_'+log_name+'.pickle', 'rb') as f:
    test_event_log = pickle.load(f)

#test_event_log['time:timestamp'] = test_event_log['time:timestamp_complete']
test_event_log['case:concept:name'] = test_event_log['case:concept:name'].astype(str)

In [ ]:
N = 1000
import conduct_evaluation


In [ ]:
drbart_model_A = DRBART(parser_dir = '../../../models/pcr/concept-name/',
                     strict_parser=True)
evaluator_A = conduct_evaluation.ConductEvaluation(drbart_model_A, SampleOutcomes_DRBART_PCR_A, {},
                                     test_event_log, n=N)
likelihoods_A = evaluator_A.sample_cases(False)

In [ ]:
exclude_A = list(filter(lambda k : k[1]==0, likelihoods_A[0].items()))
exclude_A

In [ ]:
sum([v.ln() for v in likelihoods_A[0].values()])

In [ ]:
get_pscores = lambda likelihoods : [pscore(likelihoods[1][i], likelihoods[2][k][3]).compute()[0] for i, k in enumerate(list(likelihoods[0].keys()))]

In [ ]:
np.mean(get_pscores(likelihoods_A))

In [ ]:
drbart_model_A_S = DRBART(parser_dir = '../../../models/pcr/concept-name_seconds-in-day/',
                     strict_parser=True)
evaluator_A_S = conduct_evaluation.ConductEvaluation(drbart_model_A_S, SampleOutcomes_DRBART_PCR_A_S, {},
                                     test_event_log, n=N)
likelihoods_A_S = evaluator_A_S.sample_cases(False)

In [ ]:
exclude_A_S = list(filter(lambda k : k[1]==0, likelihoods_A_S[0].items()))
exclude_A_S

In [ ]:
sum([v.ln() for v in likelihoods_A_S[0].values()])

In [ ]:
np.mean(get_pscores(likelihoods_A_S))

In [ ]:
drbart_model_A_S_D = DRBART(parser_dir = '../../../models/pcr/concept-name_seconds-in-day_day-of-week/',
                     strict_parser=True)
evaluator_A_S_D = conduct_evaluation.ConductEvaluation(drbart_model_A_S_D, SampleOutcomes_DRBART_PCR_A_S_D, {},
                                     test_event_log, n=N)
likelihoods_A_S_D = evaluator_A_S_D.sample_cases(False)

In [ ]:
exclude_A_S_D = list(filter(lambda k : k[1]==0, likelihoods_A_S_D[0].items()))
exclude_A_S_D

In [ ]:
sum([v.ln() for v in likelihoods_A_S_D[0].values()])

In [ ]:
np.mean(get_pscores(likelihoods_A_S_D))

In [ ]:
drbart_model_A_S_AC = DRBART(parser_dir = '../../../models/pcr/concept-name_seconds-in-day_activity-count/',
                     strict_parser=True)
evaluator_A_S_AC = conduct_evaluation.ConductEvaluation(drbart_model_A_S_AC, SampleOutcomes_DRBART_PCR_A_S_AC, {},
                                     test_event_log, n=N)
likelihoods_A_S_AC = evaluator_A_S_AC.sample_cases(False)

In [ ]:
cases = test_event_log['case:concept:name'].unique()
case_data = dict([(case_name, evaluator_A_S_AC.get_case_data(case_name)) for case_name in cases])

In [ ]:
exclude_A_S_AC = list(filter(lambda k : k[1]==0, likelihoods_A_S_AC[0].items()))
exclude_A_S_AC

In [ ]:
sum([v.ln() for v in likelihoods_A_S_AC[0].values()])

In [ ]:
np.mean(get_pscores(likelihoods_A_S_AC))

In [ ]:
results = {
    'drbart_model_A' : likelihoods_A,
    'drbart_model_A_S' : likelihoods_A_S,
    'drbart_model_A_S_D' : likelihoods_A_S_D,
    'drbart_model_A_S_AC' : likelihoods_A_S_AC
}
with open('./pcr_dr_bart_evaluation_'+log_name+'.pickle', 'wb') as handle:
    pickle.dump(results, handle)

In [ ]:
with open('./pcr_dr_bart_evaluation_'+log_name+'.pickle', 'rb') as handle:
    results = pickle.load(handle)

In [ ]:
likelihoods_A_S_D = results['drbart_model_A_S_D']